In [ ]:
import unittest
import pandas as pd
import trustyai as trustyai
from platform import python_version
from trustyai.metrics.fairness.group import statistical_parity_difference
from trustyai.model import output

class TestTrustyaiNotebook(unittest.TestCase):

    def test_python_version(self): 
        expected_major_minor = '3.11' # Set the expected version (x.y) 
        actual_major_minor = '.'.join(python_version().split('.')[:2])
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect version")

    def test_trustyai_version(self): 
        expected_major_minor = '0.5' # Set the expected version (x.y) 
        actual_major_minor = '.'.join(trustyai.__version__.split('.')[:2]) 
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect version")

    def test_fairnessmetrics(self):
        url_unbiased = "https://raw.githubusercontent.com/opendatahub-io/notebooks/main/jupyter/trustyai/ubi9-python-3.9/test/income-unbiased.csv"
        nobias = pd.read_csv(url_unbiased, index_col=False)
        
        nobias = pd.read_csv(url_unbiased, index_col=False)
        nobias.groupby(['gender', 'income'])['income'].count()
        nobias.groupby(['gender', 'income'])['income'].count().unstack().plot.bar()
        
        nobias_privileged = nobias[nobias.gender == 1]
        nobias_unprivileged = nobias[nobias.gender == 0]
        favorable = output("income", dtype="number", value=1)
        score = statistical_parity_difference(privileged=nobias_privileged,
                                              unprivileged=nobias_unprivileged,
                                              favorable=[favorable])
        self.assertTrue(score >= 0.0036255104824703954)  
        print("On the test_fairness_metrics test case the statistical_parity_difference score for this dataset is between the threshold [-0.1,0.1], which classifies the model as reasonably fair.")
        
    def test_datafairness(self):
        url_biased = "https://raw.githubusercontent.com/opendatahub-io/notebooks/main/jupyter/trustyai/ubi9-python-3.9/test/income-biased.csv"
        bias = pd.read_csv(url_biased, index_col=False)
        
        # Perform the data manipulations 
        grouped_counts = bias.groupby(['gender', 'income'])['income'].count()
        unstacked_counts = bias.groupby(['gender', 'income'])['income'].count().unstack()
        unstacked_counts.plot.bar()

        bias_privileged = bias[bias.gender == 1]
        bias_unprivileged = bias[bias.gender == 0]
        favorable = output("income", dtype="number", value=1)
        score = statistical_parity_difference(privileged=bias_privileged,
                                              unprivileged=bias_unprivileged,
                                              favorable=[favorable])
        self.assertTrue(score <= -0.15670061634672994)
        print("On the test_bias_metrics test case the statistical_parity_difference score for this dataset, as expected, is outside the threshold [-0.1,0.1], which classifies the model as unfair.")
    
suite = unittest.TestLoader().loadTestsFromTestCase(TestTrustyaiNotebook)
unittest.TextTestRunner().run(suite)